<a href="https://colab.research.google.com/github/rockey1006/verification/blob/main/0201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/facenet_weights.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/root/.deepface/weights/')

In [43]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/LFW0118/batch009.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/')

In [7]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/batchwrong.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/')

In [8]:

import sys
import base64
import cv2
import time
from deepface import DeepFace
from deepface.commons import functions, realtime, distance as dst
from deepface.basemodels import VGGFace, OpenFace, Facenet, FbDeepFace, DeepID
import numpy as np
import imageio

In [9]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
facenet_model = Facenet.loadModel()

def load_img2(img, do_random_crop, do_random_flip, image_size, do_prewhiten=True):
	images = np.zeros((1, image_size, image_size, 3))
	# scale pixel values
	face_pixels = img.astype('float32')
	# standardize pixel values across channels (global)
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	# transform face into one sample
	images[:,:,:,:] = face_pixels
	return images

In [10]:
def verify_distance(img1, img2):   # assumes image is in 160x160 format
	img1 = load_img2(img1, False, False, 160)
	img2 = load_img2(img2, False, False, 160)
	img1_representation = facenet_model.predict(img1)[0,:]
	img2_representation = facenet_model.predict(img2)[0,:]
	#distance1 = dst.findCosineDistance(img1_representation, img2_representation)
	#distance2 = dst.findEuclideanDistance(img1_representation, img2_representation)
	distance3 = dst.findEuclideanDistance(dst.l2_normalize(img1_representation), dst.l2_normalize(img2_representation))
	#result = str(distance1) + ' ' + str(distance2) + ' ' + str(distance3)
	return distance3
from skimage import io

In [11]:

def facedetect_image(img_file):  # detects face and returns 160x160 image around face
  imagePath = img_file
  image = io.imread(imagePath)
  margin = 3  # 3 pixel extra around detected face
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = faceCascade.detectMultiScale(
		gray,
		scaleFactor=1.3,
		minNeighbors=3,
		minSize=(30, 30)
	)
 
  for (x, y, w, h) in faces:
		#cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    roi_color = image[y-margin:y + h+margin, x-margin:x + w+margin]
    dsize=(160,160)
    global imface
    try:
      imface = cv2.resize(roi_color, dsize, interpolation = cv2.INTER_AREA)
    except:
      continue
    imface = cv2.resize(roi_color, dsize, interpolation = cv2.INTER_AREA)
    
		#print("[INFO] Object found. Saving locally.")
		#cv2.imwrite(str(w) + str(h) + '_faces.jpg', imface1)
    encoded_string = imface 
    break
  return imface

In [ ]:
def main():
  import os 

  g = os.walk(r"/content/batch009/batch29/") 
  srcPath='/content/batch009_verified/Michael_Powell_0004_0'
  if not os.path.exists(srcPath):
        os.makedirs(srcPath)
  img1 = facedetect_image('/content/batchwrong/batch009/Michael_Powell_0004_0.png/')


  for path,dir_list,file_list in g:  
    for file_name in file_list:  
      try:
        img2 = facedetect_image(os.path.join(path, file_name))
      except:
        continue
      img2 = facedetect_image(os.path.join(path, file_name))
     
      
      res = verify_distance(img1,img2)
      #print("euclidean_l2 distance: ",res)
      if res<1.2:
        print("euclidean_l2 distance: ",res)
        path1 = os.path.join(path,file_name)


        path2 = os.path.join(srcPath+'/'+str(res))
     
        with open(path1, 'rb') as rstream:
          container = rstream.read()
          with open(path2, 'wb') as wstream:
            wstream.write(container)

if __name__ == "__main__":
    sys.exit(int(main() or 0))

In [ ]:
!zip -r /content/drive/MyDrive/LFW_VERIFIED0128/batch007_verified.zip /content/batch007_verified
